# Mercator - Isopycnais

#### Packages

In [2]:
import xarray as xr
import numpy  as np

from gsw import p_from_z
from gsw import pot_rho_t_exact as prho

import glob
import os

import warnings
warnings.filterwarnings("ignore")

#### Importing Dataset

In [3]:
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
ds = xr.open_mfdataset(filenames)
_, index = np.unique(ds['time'], return_index=True)
ds = ds.isel(time=index)

#### Calculating Pressure and Potential Density

In [4]:
ds = ds.assign(pressure=lambda dsmerged: p_from_z(-dsmerged['depth'],dsmerged['latitude'],sea_surface_geopotential=dsmerged['zos']))
ds = ds.transpose('time','depth','latitude','longitude')
ds['pressure'].attrs["units"] = "dbar"
ds['pressure'].attrs["unit_long"] = "Decibar"
ds['pressure'].attrs["long_name"] = "Pressure"
ds['pressure'].attrs["standard_name"] = "pressure"

ds = ds.assign(pot_rho=lambda ds: prho(ds['so'],ds['thetao'],ds['pressure'],0))
ds['pot_rho'].attrs["units"] = "kg/m^3"
ds['pot_rho'].attrs["unit_long"] = "Kilograms per Cubic Meter"
ds['pot_rho'].attrs["long_name"] = "Potential Density"
ds['pot_rho'].attrs["standard_name"] = "potential_density"

#### Calculating TW Depths

    Profundidade mínima da ACAS 
        Limite ACAS-TW

In [5]:
depth = ds['depth'].to_dataframe()
depth.reset_index(drop=True,inplace=True,)
ds['pot_rho'] = ds.pot_rho.fillna(-999999) # mask nan for TW Depth

In [6]:
TW = ds.map(lambda p: ds.pot_rho <= 1025.7) # Return array with boolean % similiar to where
tw = TW.pot_rho.values

isopycnal = np.zeros((TW.time.size,TW.latitude.size,TW.longitude.size)) 
 # loop through time indices
for ti in np.arange(0,TW.time.size,1):
    # loop through lat,lon indices
    for yi in np.arange(0,TW.latitude.size,1):        
        for xi in np.arange(0,TW.longitude.size,1):
            # Condition for TW
            iso_depth = []
            for i in np.arange(29):
                if (i < 28) and ((tw[ti][i][yi][xi] == True) and (tw[ti][i+1][yi][xi] == False)):
                    iso_depth = depth.loc[i]
                    break
                else:
                    iso_depth = np.nan          
            

            isopycnal[ti,yi,xi] = iso_depth

In [7]:
ds['TW_depth'] = (['time','latitude','longitude'],isopycnal)
ds['TW_depth'].attrs["units"] = "m"
ds['TW_depth'].attrs["unit_long"] = "meters"
ds['TW_depth'].attrs["long_name"] = "Isopycnal Depth"
ds['TW_depth'].attrs["standard_name"] = "isopycnal_depth"

#### Exporting TW_Depth as NC

In [9]:
ds['TW_depth'].to_netcdf('D:/00_Masters/01_Dados/TW_depth.nc')

-----

######## Exporting mask to find stations

In [ ]:
vv = month_climatology.to_dataframe() 
mask = vv['TW_depth'] < 100
idx = pd.IndexSlice
vv[mask].to_csv('D:/00_Masters/01_Dados/TW_Depths_INTEREST.csv',sep=',')